In [135]:
import numpy as np
import scipy as sp
import pandas as pd
import itertools

Обозначим все узлы дорожной сети числами от 0 до 56, вокзал будет 0. Введем матрицу $57\times 57$ стоимости перемещения $A_{ij}$, равную стоимости перемещения между узлами с номерами $i,j$. Введем также вектор $B_j$, который соответствует числу людей в экскурсионной группе для узла $j$. За множество $S$ обозначим номера всех достопримечательностей.

In [55]:
MAX_INT = 100
nodes_frame = pd.read_csv("task-2-nodes.csv", header=None, index_col=0).sort_values(1)
edges_frame = pd.read_csv("task-2-adjacency_matrix.csv", index_col=0, na_values=["-"])

# Sorting nodes such that initial is 0, and all crossings are at the beginning
initial_node = "Вокзал"
node_names = nodes_frame.index.tolist()
node_names.remove(initial_node)
node_names.insert(0, initial_node)
assert len(node_names) == 57 and node_names[0]==initial_node, "Sonething goes wrong with reordering initial node"

A = edges_frame.reindex(index=node_names, columns=node_names).to_numpy(na_value=MAX_INT, copy=True).astype(int)
B = nodes_frame.reindex(index=node_names).to_numpy(na_value=MAX_INT, copy=True).astype(int).reshape((-1))
sights_start = np.searchsorted(B, 0, side='right')  # Start of nodes which should be visited only once

In [46]:
A

array([[  0, 100, 100, ..., 100, 100, 100],
       [100,   0,   5, ..., 100, 100, 100],
       [100,   5,   0, ..., 100, 100, 100],
       ...,
       [100, 100, 100, ...,   0, 100,   3],
       [100, 100, 100, ..., 100,   0, 100],
       [100, 100, 100, ...,   3, 100,   0]])

In [205]:
B

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 9, 9, 9])

In [204]:
len(list(*np.where(B)))

35

Поскольку имеется ограничение в 15 тактов, заведем для каждого автобуса векторы как в отборочной задаче, однако допустим, чтобы вершины повторялись. Тогда нам потребуется отжиг $15 \cdot 14 \cdot 57 = 11970$ кубитов. Это довольно-таки много, и асимптотика также плохая.

По сути, минимально нам требуется закодировать маршрут каждого автобуса. Для этого достаточно указать 14 промежуточных вершин, подразумевая что 0-ая и 15-ая это вокзал. Как минимум нам требуется $15 \cdot 14 = 210$ дискретных переменных от 0 до 56. В лучшем случае, используя двочную кодировку нам потребуется $\lceil \log_2 57 \rceil = 6$ кубитов, что дает вменяемый размер задачи $15 \cdot 14 \cdot 6 = 1260$. Проблема может возникнуть на этапе вычисления выражений
$$
a_{v^b_i, v^b_j} = \sum_{\alpha, \beta} a_{\alpha, \beta}\delta(\alpha, v^b_i) \delta(\beta, v^b_j)
$$
Если $\alpha = \sum_{i=0}^{5} \alpha[i] 2^{i}$, а $v$ закодировано в виде 6 бит так же ($v = \sum_{i=0}^5 v[i]2^i$), имеем
$$
\delta(\alpha, v) = \prod_{i=0}^{5} (v[i]\cdot \alpha[i] + (1-v[i])\cdot(1 - \alpha[i]))
$$

## Прямолинейный метод кодирования

**Важно:** мы будем предполагать, что автобусу запрещено стоять на достопримечательности.

Заведем бинарный вектор $v^b_{ij}$, где $b\in\{0,\dots, 14\}$, $i\in \{1,\dots, 14\}$ и $j\in\{0,\dots, 57\}$, и
$$
v^b_{ij} = \begin{cases}
    1, & \text{если $b$-й автобус на $i$ такте перемещается в вершину $j$}, \\
    0, & \text{иначе}.
\end{cases}
$$

В таком случае имеются следующее естественное ограничение:
$$
\forall b, i \hookrightarrow \sum_{j} v^b_{ij} = 1,
$$
выражающее собой тот факт, что на $i$-м такте автобус находится хотя бы в одной вершине.

Его функция потерь есть
$$
L_v := \sum_{b,i} \left( \sum_j v^b_{ij} - 1 \right)^2 = \sum_{b,i} \left( \sum_{j,j'} v^b_{ij}v^b_{ij'} - 2 \sum_j v^b_{ij} + 1 \right)
$$

Далее определим условия в виде ограничений.
1. Автобусов уже 15. Чтобы не привысить вместимость, необходимо выполнить условие
$$
\forall b \hookrightarrow \sum_{j} B_j \sum_{i} v^b_{ij} \leq 10
$$
Вводим дополнительные переменные - число занятых мест в виде one-hot-encoding, $y^b_k = 1$ если в автобусе $b$ занято $k$ мест, $k=0,\dots,10$. Ограничения такой кодировки есть
$$
\forall b \hookrightarrow \sum_k y^b_k = 1,
$$
что соответствует потерям
$$
L_y := \sum_b \left( \sum_{k} y^b_k  - 1 \right)^2 = \sum_b \left( \sum_{k,k'} y^b_k y^b_{k'} - 2\sum_k y^b_k + 1 \right) .
$$
Тогда функция потерь этого пункта есть
$$
L_1 := \sum_b \left( \sum_{i,j} B_j v^{b}_{ij} - \sum_k k y^b_k \right)^2 = \sum_b \left( \sum_{i,j,i',j'} B_jB_{j'} v^{b}_{ij}v^{b}_{i'j'} + \sum_{k,k'} kk' y^b_k y^b_{k'} - 2\sum_{i,j,k} B_j k v^{b}_{ij} y^b_k \right)
$$

2. Достопримечательности должны быть посещены ровно 1 раз (и нельзя в них стоять)
$$
\forall j \in S \hookrightarrow \sum_{b,i} v^b_{ij} = 1
$$
Соответственно потери есть
$$
L_2 := \sum_{j\in S} \left(\sum_{b,i} v^b_{ij} - 1 \right)^2 = \sum_{j\in S} \left(\sum_{b,i,b',i'} v^{b}_{ij}v^{b'}_{i'j} - 2\sum_{b,i} v^b_{ij} + 1 \right)
$$

3. На узле (кроме вокзала) не более 1 машины
$$
\forall i, j \neq 0 \hookrightarrow \sum_{b} v^b_{i,j} \leq 1.
$$
Это эффективно соответствует функции потерь
$$
L_3 := \sum_{i,j\neq 0} \sum_{b_1\neq b_2} v^{b_1}_{ij} v^{b_2}_{ij}
$$

Минимизировать требуется функцию (старт и финиш на вокзале):
$$
H := \sum_{b} \left( \sum_j A_{0j}v^b_{1j} + \sum_{i=1}^{13} \sum_{j,j'} A_{jj'} v^b_{ij} v^b_{(i+1)j'} + \sum_j A_{j0} v^b_{14j} \right)
$$

In [169]:
NUM_BUSES = 15
MAX_PEOPLE = 10
MAX_TACTS = 15
NUM_TACTS = MAX_TACTS - 1
NUM_NODES = len(node_names)
NUM_PEOPLE = MAX_PEOPLE + 1

# Array will be encoded as {v[b,i,j], y[b,k]}
# b = 0,...,NUM_BUSES-1
# i = 1,...,NUM_TACTS-1
# j = 0,...,NUM_NODES-1
# k = 0,...,MAX_PEOPLE

def get_index(inp : list) -> int:
    """
    Returns integer index in big array of element v,b,i,j or y,b,k
    """
    var = inp
    if var[0] == "v":
        return (MAX_TACTS-1)*NUM_NODES*int(var[1]) + NUM_NODES*(int(var[2]) - 1) + int(var[3])
    elif var[0] == "y":
        return NUM_BUSES*NUM_TACTS*NUM_NODES + (MAX_PEOPLE + 1)*int(var[1]) + int(var[2])

In [187]:
SHAPE = NUM_BUSES*NUM_TACTS*NUM_NODES + NUM_BUSES*NUM_PEOPLE
H = np.zeros((SHAPE, SHAPE))
for b, j in itertools.product(range(NUM_BUSES), range(NUM_NODES)):
    ind = get_index(["v",b,1,j])
    H[ind, ind] += A[0,j]
    ind = get_index(["v",b,NUM_TACTS,j])
    H[ind, ind] += A[j,0]

for b, i, j, jp in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS-1), range(NUM_NODES), range(NUM_NODES)):
    H[get_index(["v",b,i,j]), get_index(["v",b,i+1,jp])] += A[j,jp]

In [177]:
Lv = np.zeros((SHAPE, SHAPE))
for b, i, j in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS), range(NUM_NODES)):
    Lv[get_index(["v",b,i,j]), get_index(["v",b,i,j])] -= 2
for b, i, j, jp in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS), range(NUM_NODES), range(NUM_NODES)):
    Lv[get_index(["v",b,i,j]), get_index(["v",b,i,jp])] += 1

In [190]:
Ly = np.zeros((SHAPE, SHAPE))
for b, k in itertools.product(range(NUM_BUSES), range(NUM_PEOPLE)):
    ind = get_index(["y", b, k])
    Ly[ind, ind] -= 2
    
for b, k, kp in itertools.product(range(NUM_BUSES), range(NUM_PEOPLE), range(NUM_PEOPLE)):
    Ly[get_index(["y", b, k]), get_index(["y", b, kp])] += 1

In [192]:
L1 = np.zeros((SHAPE, SHAPE))
for b in range(NUM_BUSES):
    for i,ip,j,jp in itertools.product(range(1,MAX_TACTS), range(1,MAX_TACTS), 
                                       range(sights_start, NUM_NODES), range(sights_start, NUM_NODES)):
        L1[get_index(["v", b, i, j]), get_index(["v", b, ip, jp])] += B[j]*B[jp]
    
    for k,kp in itertools.product(range(1, NUM_PEOPLE), range(1, NUM_PEOPLE)):
        L1[get_index(["y", b, k]), get_index(["y", b, kp])] += k*kp
        
    for i,j,k in itertools.product(range(1, MAX_TACTS), range(sights_start, NUM_NODES), range(1, NUM_PEOPLE)):
        L1[get_index(["v", b, i, j]), get_index(["y", b, k])] -= 2 * B[j] * k

In [194]:
L2 = np.zeros((SHAPE, SHAPE))
for j in range(sights_start, NUM_NODES):
    for b,i in itertools.product(range(NUM_BUSES), range(1, MAX_TACTS)):
        ind = get_index(["v", b, i, j])
        L2[ind, ind] -= 2
        for bp,ip in itertools.product(range(NUM_BUSES), range(1, MAX_TACTS)):
            ind2 = get_index(["v", bp, ip, j])
            L2[ind, ind2] += 1

In [195]:
L3 = np.zeros((SHAPE, SHAPE))
for i,j in itertools.product(range(1, MAX_TACTS), range(1, NUM_NODES)):
    for b,bp in itertools.combinations(range(NUM_BUSES), 2):
        L3[get_index(["v", b, i, j]), get_index(["v", bp, i, j])] += 1

In [196]:
NUM_SIGHTS = NUM_NODES - sights_start
FREE_TERM = NUM_BUSES * NUM_TACTS + NUM_BUSES + NUM_SIGHTS

In [ ]:
Q = sp.sparse.coo_array()

In [207]:
sp.sparse.save_npz("test.npz", )

In [208]:
tst = sp.sparse.coo_array(np.arange(4).reshape((2,2)))

In [188]:
ARR_SHAPE_V = (NUM_BUSES, NUM_TACTS, NUM_NODES)
SHAPE_V = NUM_BUSES*NUM_TACTS*NUM_NODES
ARR_SHAPE_Y = (NUM_BUSES, NUM_PEOPLE)
SHAPE_Y = NUM_BUSES*NUM_PEOPLE

Hij = np.zeros((NUM_TACTS, NUM_NODES, NUM_TACTS, NUM_NODES))
Hij[0,:,0,:] = np.diag(A[0,:])    # first term
Hij[-1,:,-1,:] = np.diag(A[:,0])  # last term
for i in range(NUM_TACTS - 1):
    Hij[i,:,i+1,:] += A
Ht = np.reshape(np.einsum("ab,ijkl->aijbkl", np.eye(NUM_BUSES), Hij), (SHAPE_V, SHAPE_V))

np.reshape(Hij, (NUM_TACTS*NUM_NODES, NUM_TACTS*NUM_NODES))

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0., 100.,   0., ...,   0.,   0.,   0.],
       [  0.,   0., 100., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 100.,   0.,   0.],
       [  0.,   0.,   0., ...,   0., 100.,   0.],
       [  0.,   0.,   0., ...,   0.,   0., 100.]])

In [189]:
np.max(H[:SHAPE_V, :SHAPE_V]-Ht)

0.0

In [154]:
SHAPE_V

11970

In [93]:
tst = np.array([[1,2], [3,4]])

In [95]:
tst[:,1] = 0

In [96]:
tst

array([[1, 0],
       [3, 0]])

In [100]:
(*ARR_SHAPE_V, *ARR_SHAPE_V)

(15, 14, 57, 15, 14, 57)

In [123]:
tst = np.arange(36)

In [127]:
tst2 = tst.reshape((2,3,2,3))

In [128]:
tst2

array([[[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17]]],


       [[[18, 19, 20],
         [21, 22, 23]],

        [[24, 25, 26],
         [27, 28, 29]],

        [[30, 31, 32],
         [33, 34, 35]]]])

In [131]:
tst2.reshape((6,6))[2,:]

array([12, 13, 14, 15, 16, 17])